In [3]:
import pandas as pd
import time
import random

from bs4 import BeautifulSoup as bs
import requests
import numpy as np

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()

frescos = {
    "Frutas": "https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=",
    "Carne": "https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=",
    "Panadería":"https://www.alcampo.es/compra-online/frescos/panaderia/c/W1281?q=%3Arelevance&page=",
    # "Verduras y hortalizas":"https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/c/W1702?q=%3Arelevance&page=",
    # "Charcutería":"https://www.alcampo.es/compra-online/frescos/charcuteria/c/W15?q=%3Arelevance&page=",
    # "Pastelería":"https://www.alcampo.es/compra-online/frescos/pasteleria/c/W1282?q=%3Arelevance&page=",
    # "Sushi": "https://www.alcampo.es/compra-online/frescos/sushi/c/W140502?q=%3Arelevance&page=",
    # "Quesos": "https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=",
    # "Pescados, mariscos y moluscos": "https://www.alcampo.es/compra-online/frescos/pescados-mariscos-y-moluscos/c/W14?q=%3Arelevance&page=",
    # "Ahumados, sucedáneos gulas, anchoas": "https://www.alcampo.es/compra-online/frescos/ahumados-sucedaneos-gulas-anchoas/c/W184?q=%3Arelevance&page=",
    # "Fresco al Corte": "https://www.alcampo.es/compra-online/frescos/fresco-al-corte/c/WFrescoalcorte?q=%3Arelevance&page="
}



dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in frescos.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)

df_alcampo








Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=0
Got an attribute error on: https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=0
Category: 0 Name: 0 Image: 0 Price: 0 Stock 0
Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=1
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=2
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=3
Got an attribute error on: https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=3
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=4
No hay más ventanas
Carne https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=0
Category: 96 Name: 96 Image: 96 P

,Category,Name,Price,Stock,Image
0,Frutas,Fresón caja rosa ALCAMPO PRODUCCIÓN CONTROLADA...,"6,99 €(6,99 €/Kg)",Yes,https://www.alcampo.es/media/h24/haa/120256584...
1,Frutas,Tomate rama a granel,"2,99 €(2,99 €/Kg)",Yes,https://www.alcampo.es/media/hf8/h9f/118741086...
2,Frutas,Zumo de naranja natural recién exprimido ALCAM...,"5,59 €(5,59 €/Litro)",Yes,https://www.alcampo.es/media/h82/ha8/115742002...
3,Frutas,"Ciruela roja, bandeja de 500 g.","2,69 €(5,38 €/Kg)",Yes,https://www.alcampo.es/media/h9d/hf7/889773293...
4,Frutas,Manzana lavada y cortada FLORETTE 80 gramos,"0,71 €(8,87 €/Kg)",Yes,https://www.alcampo.es/media/h18/ha8/895977596...
...,...,...,...,...,...
782,Panadería,"Triangulo de bácon y queso, 2uds.","1,95 €(8,66 €/Kg)",No,https://www.alcampo.es/media/h8d/h3d/923612161...
783,Panadería,"Pan rallado con ajo (1%) y perejil (0,4%), 250g.","1,59 €(0,00 €/Kg)",No,https://www.alcampo.es/media/h8d/hdb/925632646...
784,Panadería,"Empanadilla bocata de carne, 120g.","1,29 €(10,75 €/Kg)",No,https://www.alcampo.es/media/h4e/h11/921102240...
785,Panadería,"Empanadilla media luna latina de atún, 2uds. 2...","2,99 €(13,59 €/Kg)",Yes,https://www.alcampo.es/media/hc7/h37/975398466...


In [4]:
df_alcampo['Shop'] = 'Alcampo'

today = pd.Timestamp.now()
df_alcampo['Date'] = today
df_alcampo

df_alcampo.to_csv("../data/alcampo.csv", mode='a', index=False, header=False)

read_file = pd.read_csv ("../data/alcampo.csv")
read_file.to_excel ("../data/alcampo.xlsx", index = None, header=True)

alcampo = pd.read_csv("../data/alcampo.csv")
print("File saved",today)
alcampo

File saved 2023-03-09 17:43:21.448537


,Category,Name,Price,Stock,Image,Shop,Date
0,Veganos,Bebida de avena con alto contenido de calcio y...,"5,94 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hbc/hc7/114853804...,Alcampo,2023-03-08 19:51:25.159831
1,Veganos,"Bebida de soja sin gluten, enriquecida con cal...","5,34 €(0,89 €/Litro)",Yes,https://www.alcampo.es/media/h2b/hc3/107936543...,Alcampo,2023-03-08 19:51:25.159831
2,Veganos,Bebida de avena con alto contenido de calcio y...,"0,99 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hc3/h79/109357267...,Alcampo,2023-03-08 19:51:25.159831
3,Veganos,"Bebida de almendras 100% vegetal, sin azúcar n...","2,10 €(2,10 €/Litro)",Yes,https://www.alcampo.es/media/h31/h28/114171706...,Alcampo,2023-03-08 19:51:25.159831
4,Veganos,Bebida de avena ALCAMPO ECOLÓGICO brick de 1 l...,"11,22 €(0,18 €/100ml)",Yes,https://www.alcampo.es/media/hb5/h66/111936354...,Alcampo,2023-03-08 19:51:25.159831
...,...,...,...,...,...,...,...
20747,Panadería,"Triangulo de bácon y queso, 2uds.","1,95 €(8,66 €/Kg)",No,https://www.alcampo.es/media/h8d/h3d/923612161...,Alcampo,2023-03-09 17:43:21.448537
20748,Panadería,"Pan rallado con ajo (1%) y perejil (0,4%), 250g.","1,59 €(0,00 €/Kg)",No,https://www.alcampo.es/media/h8d/hdb/925632646...,Alcampo,2023-03-09 17:43:21.448537
20749,Panadería,"Empanadilla bocata de carne, 120g.","1,29 €(10,75 €/Kg)",No,https://www.alcampo.es/media/h4e/h11/921102240...,Alcampo,2023-03-09 17:43:21.448537
20750,Panadería,"Empanadilla media luna latina de atún, 2uds. 2...","2,99 €(13,59 €/Kg)",Yes,https://www.alcampo.es/media/hc7/h37/975398466...,Alcampo,2023-03-09 17:43:21.448537
